In [1]:
import pandas as pd

In [2]:
news = pd.read_csv("C:/Ангелина/Data Sience/5 четверть/ML in business/2/Lection2/materials.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [3]:
users = pd.read_csv("C:/Ангелина/Data Sience/5 четверть/ML in business/2/Lection2/users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


###  Получаем векторные представления новостей

In [4]:
!pip install razdel pymorphy2

In [5]:
import re
import numpy as np
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize 
import pymorphy2  

In [6]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

151


In [8]:
with open('C:/Ангелина/Data Sience/5 четверть/ML in business/2/Lection2/stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
    
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [9]:
def clean_text(text):
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)
    
    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):    
    if not isinstance(text, str):
        text = str(text)
    
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': 
            w = w[1:]
        if len(w) > 1: 
            if w in cache: 
                words_lem.append(cache[w])
            else: 
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords = [i for i in words_lem if not i in stopword_ru] 
    
    return words_lem_without_stopwords

In [10]:
from tqdm import tqdm
tqdm.pandas()

news['title'] = news['title'].progress_apply(lambda x: clean_text(x))

  0%|                                                                                        | 0/27000 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp/ipykernel_17636/3272013200.py:9: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
100%|███████████████████████████████████████████████████████████████████████████| 27000/27000 [00:29<00:00, 928.86it/s]


In [11]:
news['title'].iloc[:10]

0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
2    форвард авангарда томаш заборский прокомментир...
3    главный тренер кубани юрий красножан прокоммен...
4    решением попечительского совета владивостокско...
5    ио главного тренера вячеслав буцаев прокоммент...
6    запорожский металлург дома потерпел разгромное...
7    сборная сша одержала победу над австрией со сч...
8    бывший защитник сборной россии дарюс каспарайт...
9    полузащитник цска зоран тошич после победы над...
Name: title, dtype: object

In [12]:
news['title'] = news['title'].progress_apply(lambda x: lemmatization(x))

100%|███████████████████████████████████████████████████████████████████████████| 27000/27000 [03:28<00:00, 129.77it/s]


In [13]:
texts = list(news['title'].values)

common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [14]:
N_topic = 15  

In [15]:
from gensim.models import LdaModel

lda = LdaModel(common_corpus, num_topics=N_topic, id2word=common_dictionary, passes=10) 

In [16]:
from gensim.test.utils import datapath

temp_file = datapath("model.lda")
lda.save(temp_file)

In [17]:
lda = LdaModel.load(temp_file)

In [18]:
other_texts = list(news['title'].iloc[:3])
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(0, 0.023698315),
 (1, 0.35895076),
 (2, 0.035001792),
 (10, 0.40742698),
 (11, 0.16003396)]

In [19]:
x = lda.show_topics(num_topics=N_topic, num_words=7, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

for topic, words in topics_words:
    print(f"topic_{topic}: " + " ".join(words))

topic_0: продолжительность конструкция грузия таиланд литва латвия дания
topic_1: который год день время это человек город
topic_2: это россия украина который российский мочь заявить
topic_3: год ребёнок дело суд который свой это
topic_4: торговый одежда ирак сон диапазон post дикий
topic_5: год компания млн рубль млрд рынок рост
topic_6: выяснить квартира миссия захватить высота полоса выделить
topic_7: исследование смерть статья автор рак продукция препарат
topic_8: ракета система космический корабль аппарат новый технология
topic_9: президент правительство россия год глава закон который
topic_10: городской игра команда сайт мероприятие клуб хороший
topic_11: это год который мочь учёный человек весь
topic_12: военный россия сила армия российский страна войско
topic_13: человек женщина мужчина погибнуть полиция здание уголовный
topic_14: сша американский самолёт полёт вашингтон американец пилот


векторное представление новости

In [20]:
def get_lda_vector(lda, text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]

    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(N_topic):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [21]:
get_lda_vector(lda, news['title'].iloc[0])

array([0.        , 0.        , 0.30960575, 0.22994928, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.34809425,
       0.10535478, 0.        , 0.        , 0.        , 0.        ])

In [22]:
topic_matrix = pd.DataFrame([get_lda_vector(lda, text) for text in news['title'].values])
topic_matrix.columns = [f'topic_{i}' for i in range(N_topic)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+[f'topic_{i}' for i in range(N_topic)]]
topic_matrix.head()

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
0,6,0.000000,0.000000,0.309608,0.229948,0.0,0.000000,0.0,0.0,0.000000,0.348093,0.105355,0.000000,0.0,0.0,0.000000
1,4896,0.000000,0.146676,0.193206,0.139839,0.0,0.000000,0.0,0.0,0.128391,0.000000,0.375196,0.000000,0.0,0.0,0.000000
2,4897,0.023698,0.358888,0.035138,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.407392,0.159996,0.0,0.0,0.000000
3,4898,0.000000,0.000000,0.148193,0.000000,0.0,0.038222,0.0,0.0,0.000000,0.000000,0.276765,0.452221,0.0,0.0,0.076546
4,4899,0.000000,0.000000,0.392720,0.120235,0.0,0.000000,0.0,0.0,0.000000,0.306615,0.159947,0.000000,0.0,0.0,0.000000


### Следующий шаг - векторные представления пользователей

In [23]:
users.head(5)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"
3,u101138,"[5933, 6186, 5055, 6977, 5206, 488389]"
4,u108248,"[707, 1144, 2532, 2928, 3133, 324592]"


In [24]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[f'topic_{i}' for i in range(N_topic)]].values))

In [25]:
doc_dict[293672]

array([0.        , 0.1893031 , 0.        , 0.05659429, 0.        ,
       0.25158021, 0.        , 0.        , 0.        , 0.30016249,
       0.02026217, 0.16904524, 0.        , 0.        , 0.        ])

In [26]:
def get_user_embedding(user_articles_list, doc_dict, func):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, axis=0)
    return user_vector

In [27]:
user_articles_list = users['articles'].iloc[33]

get_user_embedding(user_articles_list, doc_dict, np.median)

array([0.        , 0.06400611, 0.11673371, 0.09133679, 0.        ,
       0.        , 0.        , 0.0062589 , 0.        , 0.12361932,
       0.        , 0.0575522 , 0.03525028, 0.        , 0.02297945])

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [28]:
FUNC = np.mean

In [29]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
0,u105138,0.000000,0.220464,0.066136,0.140484,0.005074,0.041930,0.000000,0.011284,0.002407,0.139680,0.046997,0.218897,0.054689,0.025300,0.012886
1,u108690,0.002492,0.145149,0.334557,0.124627,0.000000,0.057039,0.000000,0.010555,0.004699,0.066670,0.004669,0.149496,0.022274,0.052350,0.012884
2,u108339,0.002413,0.320340,0.192001,0.109651,0.000000,0.050700,0.006311,0.013334,0.005607,0.113773,0.000000,0.049538,0.047410,0.068824,0.010746


In [30]:
target = pd.read_csv("C:/Ангелина/Data Sience/5 четверть/ML in business/2/Lection2/users_churn.csv")
target.head(5)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0
3,u103439,0
4,u104300,0


In [31]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,churn
0,u105138,0.000000,0.220464,0.066136,0.140484,0.005074,0.041930,0.000000,0.011284,0.002407,0.139680,0.046997,0.218897,0.054689,0.025300,0.012886,0
1,u108690,0.002492,0.145149,0.334557,0.124627,0.000000,0.057039,0.000000,0.010555,0.004699,0.066670,0.004669,0.149496,0.022274,0.052350,0.012884,1
2,u108339,0.002413,0.320340,0.192001,0.109651,0.000000,0.050700,0.006311,0.013334,0.005607,0.113773,0.000000,0.049538,0.047410,0.068824,0.010746,1


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [34]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [35]:
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.12185776, 0.22315556, 0.5793802 , 0.27619434, 0.01806091,
       0.03544999, 0.06576543, 0.04139089, 0.27179968, 0.13674323])

In [36]:
from sklearn.metrics import (f1_score, roc_auc_score, precision_score,
                             classification_report, precision_recall_curve, confusion_matrix)

In [37]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)

ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.2719918649766324, F-Score=0.747, Precision=0.670, Recall=0.845


In [38]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9686167800453516

In [39]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


In [40]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
   'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.271992,0.747292,0.669903,0.844898,0.968617


2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог)

In [41]:
FUNC = np.median

In [42]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
0,u105138,0.0,0.186635,0.000000,0.125067,0.0,0.000000,0.0,0.000000,0.0,0.167112,0.010131,0.194082,0.000000,0.000000,0.0
1,u108690,0.0,0.131870,0.288190,0.098621,0.0,0.000000,0.0,0.005827,0.0,0.061008,0.000000,0.175344,0.024191,0.034477,0.0
2,u108339,0.0,0.270780,0.131526,0.087441,0.0,0.011354,0.0,0.000000,0.0,0.114590,0.000000,0.040928,0.000000,0.059696,0.0


In [43]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,churn
0,u105138,0.0,0.186635,0.000000,0.125067,0.0,0.000000,0.0,0.000000,0.0,0.167112,0.010131,0.194082,0.000000,0.000000,0.0,0
1,u108690,0.0,0.131870,0.288190,0.098621,0.0,0.000000,0.0,0.005827,0.0,0.061008,0.000000,0.175344,0.024191,0.034477,0.0,1
2,u108339,0.0,0.270780,0.131526,0.087441,0.0,0.011354,0.0,0.000000,0.0,0.114590,0.000000,0.040928,0.000000,0.059696,0.0,1


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [45]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [46]:
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.14943122, 0.27433874, 0.49845366, 0.29444276, 0.00793831,
       0.07732095, 0.03704816, 0.01941939, 0.15250526, 0.07910381])

In [47]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)

ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.27994483666256703, F-Score=0.737, Precision=0.667, Recall=0.824


In [48]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.965807314378743

In [49]:
#metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
#metrics_df

In [50]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.271992,0.747292,0.669903,0.844898,0.968617
1,median,0.279945,0.737226,0.666667,0.824490,0.965807



3. Повторить п.2, но используя уже не медиану, а max


In [51]:
FUNC = np.max

In [52]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
0,u105138,0.000000,0.384694,0.256814,0.378213,0.030447,0.251580,0.000000,0.067701,0.014441,0.300162,0.165352,0.331637,0.206916,0.107127,0.077314
1,u108690,0.014954,0.273810,0.494973,0.342899,0.000000,0.193333,0.000000,0.029503,0.028197,0.134056,0.028015,0.266657,0.042046,0.190924,0.046861
2,u108339,0.014478,0.675057,0.459106,0.281844,0.000000,0.178586,0.037865,0.064720,0.033640,0.285294,0.000000,0.119844,0.166544,0.130720,0.049252


In [53]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,churn
0,u105138,0.000000,0.384694,0.256814,0.378213,0.030447,0.251580,0.000000,0.067701,0.014441,0.300162,0.165352,0.331637,0.206916,0.107127,0.077314,0
1,u108690,0.014954,0.273810,0.494973,0.342899,0.000000,0.193333,0.000000,0.029503,0.028197,0.134056,0.028015,0.266657,0.042046,0.190924,0.046861,1
2,u108339,0.014478,0.675057,0.459106,0.281844,0.000000,0.178586,0.037865,0.064720,0.033640,0.285294,0.000000,0.119844,0.166544,0.130720,0.049252,1


In [54]:
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [55]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [56]:
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.0668932 , 0.02330332, 0.8119054 , 0.51886911, 0.01561053,
       0.03089238, 0.03501684, 0.03146267, 0.14763108, 0.1811595 ])

In [57]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.38790043348891573, F-Score=0.710, Precision=0.722, Recall=0.698


In [58]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9476597476597476

In [59]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.271992,0.747292,0.669903,0.844898,0.968617
1,median,0.279945,0.737226,0.666667,0.824490,0.965807
2,amax,0.387900,0.709544,0.721519,0.697959,0.947660



5. Сформировать на выходе единую таблицу, сравнивающую качество 2/3 разных метода получения эмбедингов пользователей: median, max, idf_mean по метрикам roc_auc, precision, recall, f_score


In [60]:
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.271992,0.747292,0.669903,0.844898,0.968617
1,median,0.279945,0.737226,0.666667,0.824490,0.965807
2,amax,0.387900,0.709544,0.721519,0.697959,0.947660


6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

По таблице matrix_df видно, что максимальная точность (Precision) достигается при методе эмбендинга с max, но при этом отмечается минимальная полнота (Recall). При этом объединяющая в себе информацию о точности и полноте алгоритма, метрика F-score максимальна при методе эмбендинга с mean. 

Таким образом, получается, что из вариантов mean, median, max предпочтительным вариантом преобразования данных при эмбендинге является вычисление среднего арифметического. А вот вычисление max оказалось самым непродуктивным. 